# Array-assisted Phase Picker (APP) Tutorial 

### (Jiang Y R, Ning J Y, 2019)

#### Tutorial Script created by Jeremy Wong @2020.05.27

Source Code and tutorial is written based on : https://github.com/baogegeJiang/Array-assisted-Phase-Picker

Array-assisted Phase Picker is vector support machine, an trained model to conducts seismic arrival-time picking. The modified Array-assisted phase picker introduce futher packages ultilizing the output of the AI to conduct futher analysis. The trained model detect and locate the arrival of the P-arrival and S-arrival of the input seimsic traces.

### Table of content
* [1 - Demonstrate the Pre-trained Model](#first-bullet)
    * [1.1 Import Pre-trained Model from genMV3.py](#1.1)
    * [1.2 Import and organize data to feed the model](#1.2)
    * [1.3 Plot and analysis the result ](#1.3)
* [2 - Advance usage of Array-assisted Phase-picker](#second-bullet)
    * [2.1 Prerequsite for the APP programme](#2.1)
    * [2.2 Run APP](#2.2)



## Section 1 - Demonstrate the Pre-trained Model <a class="anchor" id="first-bullet"></a>
### 1.1 Import Pre-trained Model from genMV3.py <a class="anchor" id="1.1"></a>

In the folllowing, the model is imported via the script genMV3.py and updated with the weights pretrained with noise and without noise as selected. 

   - 'modelP_320000_0-2-15-with', 'modelS_320000_0-2-15-with': 
         models trained with typical noise
   - 'modelP_320000_100-2-15', 'modelS_320000_100-2-15':  
         models trained without typical noise
<figure>
<img src="img/SVM.png" height=70 />
<figcaption> Figure 1. Model Architecture of PhaseNet [Figure 5, Zhu W. & Beroza G. C.]
</figcaption>
</figure>

A simplified illustration of a SVM. It is a nonlinear classifier. The orginal data are transform to higher dimension via kernal and are classified by the support vector. Detailed can be found in the paper. 

In [1]:
#### Import Relevant Packages
from keras.utils import plot_model


### Import model from genMV3.py
from genMV3 import genModel
 
modelP = genModel()
modelP.load_weights('modelP_320000_0-2-15-with')
modelS = genModel()
modelS.load_weights('modelS_320000_0-2-15-with')

# show spec of the model
print("Input shape of the model :" + str(modelP.input_shape))
# Plot the structure of the model (output: img/modelP.png)
if False:               # Select True to plot the structure of the model
    plot_model(modelP, 
               to_file="img/modelP.png",
               show_shapes=True,
               show_layer_names=True,
               rankdir="TB", # TB - vertical plot; LR - horizontal plot
               expand_nested=True, # whether to expand nested models into clusters
               dpi=96)


Using TensorFlow backend.
/Users/jeremy/opt/anaconda3/envs/seismo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jeremy/opt/anaconda3/envs/seismo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jeremy/opt/anaconda3/envs/seismo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/User

Instructions for updating:
Colocations handled automatically by placer.
Input shape of the model :(None, 2000, 1, 3)


### 1.2 Import and organize data to feed the model <a class="anchor" id="1.2"></a>

1) The seismic data shall be organized in the following format to feed to the model:
<center> <code> (n, 2000, 1, 3) </code> </center>

   - n: data slices count
   - 2000: sampling points in the time domain 40s, 50hz
   - 3: components [E, N, Z]

2) Preprocessing the data
   - Filter the data with bandpass of [2 Hz - 15 Hz] 

   <code> st.filter("bandpass", freqmin=2, freqmax=15) </code>

   - Normalize the waveform using the following

   `inputData /= inputData.std(axis=(1,2,3), keepdims=True)` 

3) Feed the model with the data

   <code> probP = modelP.predict(inputData) </code>

   <code> probS = modelS.predict(inputData) </code>

Returns: Probabilities of P and S arrivals

#### Import sample data from HKPS
In the following, we will detect and locate the P and S arrival time of the recent local earthquakes in HK of the seismic data recorded by Po Shan Station. The data can be fetched using `obspy.clients.fdsn.Client`.

Since the sampling of the seismic trace maybe different from 50 hz, we have to resample the trace into 50hz with lenght of 40 seconds.

In [2]:
import numpy as np
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

verbose = True
# Obtain data from HKPS station
client = Client('IRIS')
t1 = UTCDateTime("2020-01-04T22:55:30")
st = client.get_waveforms("HK","HKPS","","BH*", t1, t1 + 40)
if verbose:
    print(st)

# resample, filter and normalize
st.resample(50.0)
st.filter("bandpass", freqmin=2, freqmax=15)
st.normalize()

# Extract data from obspy.trace
data = np.zeros((1,2000,1,3))
for i in range(3):
    data[0,:,0,i] = st[i].data


### Feed to the model
probP = modelP.predict(data)
probS = modelS.predict(data) 

3 Trace(s) in Stream:
HK.HKPS..BHE | 2020-01-04T22:55:30.019500Z - 2020-01-04T22:56:09.994500Z | 40.0 Hz, 1600 samples
HK.HKPS..BHN | 2020-01-04T22:55:30.019500Z - 2020-01-04T22:56:09.994500Z | 40.0 Hz, 1600 samples
HK.HKPS..BHZ | 2020-01-04T22:55:30.019500Z - 2020-01-04T22:56:09.994500Z | 40.0 Hz, 1600 samples


### 1.3 Plot and analysis the result  <a class="anchor" id="1.3"></a>

In [1]:
import matplotlib.pyplot as plt

def plot_results(st,probP,probS):
    # Plot stream with the probabilty of P ans S arrival predicted from the model
    fig = plt.figure(figsize=(16,8))
    ax = fig.add_subplot(411)
    ax.plot(st[0].times("matplotlib"), st[0].data)
    ax.xaxis_date()
    ax = fig.add_subplot(412)
    ax.plot(st[0].times("matplotlib"), st[1].data)
    ax.xaxis_date()
    ax = fig.add_subplot(413)
    ax.plot(st[0].times("matplotlib"), st[2].data)
    ax.xaxis_date()
    ax = fig.add_subplot(414)
    ax.plot(st[0].times("matplotlib"),probP[0,:,0,0])
    ax.plot(st[0].times("matplotlib"),probS[0,:,0,0])
    ax.xaxis_date()
    fig.autofmt_xdate()
    plt.show()

plot_results(st, probP, probS)




NameError: name 'st' is not defined

## 2 - Advance usage of Array-assisted Phase-net <a class="anchor" id="second-bullet"></a>

The developed version of he Phase Picker requires the following items to run the programme

### 2.1 Prerequsite for the APP programme <a class="anchor" id="2.1"></a>

#### 2.1.1 Station Information list file
net  sta  component  long/º  lat/º  elevation/m  rms_of_lon  rms_of_lat

`HK HKPS BH 104.55 31.00 0.0000 0.0000 0.0000`

    ```
    def generate_stationlist(st, stlst_file):
    """ Generate station list from obspy.stream
    """
        stlst = []
        for tr in st:
            stlst.append([tr.stats.network, tr.stats.station, tr.stats.channel,
                          tr.stats.longtitude, tr.stats.latitude, tr.stats.elevation,
                          0.0,0.0])
        with open(stlst_file, "w+") as file:
            for ls in stlst:
                file.write(" ".join(ls) + "\n")
    ```
Generate station infomration list

In [4]:
from obspy import read, Stream

##### Station list generator
def generate_stationlist(st, stlst_file):
    """ Generate station list from obspy.stream
    """
    stlst = []
    _net, _sta, _com = "", "", ""
    for tr in st:
        # Remove deplicated entry
        if _net==tr.stats.network and _sta==tr.stats.station and _com==tr.stats.channel[0:2]:
            pass
        else:
            _net = tr.stats.network
            _sta = tr.stats.station
            _com = tr.stats.channel[0:2]

            stlst.append([tr.stats.network, tr.stats.station, tr.stats.channel[0:2],
                          str(tr.stats.sac.stlo), str(tr.stats.sac.stla), str(tr.stats.sac.stel),
                          "0.0","0.0"])
    print(stlst)
    # stlst = list(dict.fromkeys(stlst))
    with open(stlst_file, "w+") as file:
        for ls in stlst:
            file.write(" ".join(ls) + "\n")
    #print("Generated %s " % stlst_file)


# Read sac in a day to generate station list
st_loc = "/Users/jeremy/OneDrive - The Chinese University of Hong Kong/cu/academic/sources/program/seismology/tmp/2020/005"
stlist_file = "station_list"
_st = read(st_loc + '/*')
st = Stream()

# Select list of channel only (exclude V.., L..)
for chan in "BH*", "HH*","EH*":
    st += _st.select(channel=chan)

# Gen station list
generate_stationlist(st, stlist_file)

[['GD', 'DOG', 'BH', '113.723', '22.875', '59.0', '0.0', '0.0'], ['HK', 'HKPS', 'BH', '114.142', '22.2776', '196.3', '0.0', '0.0'], ['GD', 'HUD', 'BH', '113.227', '23.5164', '65.0', '0.0', '0.0'], ['GD', 'SHW', 'BH', '115.37', '22.7916', '15.0', '0.0', '0.0'], ['GD', 'XFJ', 'BH', '114.657', '23.7385', '101.0', '0.0', '0.0'], ['GD', 'XNH', 'BH', '113.034', '22.5697', '86.0', '0.0', '0.0'], ['GD', 'YGJ', 'BH', '111.954', '21.8581', '20.0', '0.0', '0.0'], ['GD', 'ZHH', 'BH', '113.566', '22.2706', '50.0', '0.0', '0.0'], ['HK', 'HKPS', 'BH', '114.142', '22.2776', '196.3', '0.0', '0.0'], ['HK', 'HKCD', 'HH', '114.26', '22.2092', '40.0', '0.0', '0.0'], ['HK', 'HKLK', 'HH', '114.211', '22.5139', '90.0', '0.0', '0.0'], ['HK', 'HKOC', 'HH', '114.174', '22.302', '29.0', '0.0', '0.0'], ['HK', 'HKPS', 'HH', '114.142', '22.2776', '196.3', '0.0', '0.0'], ['HK', 'HKTB', 'HH', '114.012', '22.4863', '4.0', '0.0', '0.0'], ['HK', 'HKYN', 'HH', '114.338', '22.3776', '87.0', '0.0', '0.0'], ['MO', 'DTS', 'EH

#### 2.1.2 File path function
A funtion that return the sacfile name list according to the input (net, station, component, date).
    - net is the network name(e. g. 'XX' )
    - station is the station name(e. g. 'ABC' )
    - comp is the component name(e. g. 'BHE' )
    - YmdHMSJ is a dict contained date information(year, month, day, hour, minute, second, day num from the first day of the year)
    (e. g. {'Y': '2019', 'm': '01', 'd': '01', 'H': '00', 'M': '01', 'S': '00', 'J': '001'}).
   
Some sac files naming convection consists of the stations' location code. The following code have modified of the sac naming convection for GD network stations and specific stations with loc.
    ```
    def FileName(net, station, comp, YmdHMSJ):
        """
        Args: 
                net - station network
            station - station name
               comp - station component
            YmdHMSJ - dict of date
        """
        sacFileNames = list()
        c=comp[-1]
        if c=='Z':
            c='U'
        
        sacFileNames.append('data/'+net+'/'+station+'/'+net+'.'+YmdHMSJ['Y']+\
            YmdHMSJ['m']+YmdHMSJ['d']+'.'+station+'.'+c+'.SAC')
            
        # OR
        if net in ['GD']:
            loc = '_00'
            sacFileNames.append(sac_dir+'/'+YmdHMSJ['Y']+'/'+YmdHMSJ['J']+'/'\
                                +YmdHMSJ['Y']+YmdHMSJ['J']+"000000.00"+'.'+station+'.'+comp+loc)
        elif station == 'HKOC':
            loc = '_10'
            sacFileNames.append(sac_dir+'/'+YmdHMSJ['Y']+'/'+YmdHMSJ['J']+'/'\
                                +YmdHMSJ['Y']+YmdHMSJ['J']+"000000.00"+'.'+station+'.'+comp+loc)        
        else:
            sacFileNames.append(sac_dir+'/'+YmdHMSJ['Y']+'/'+YmdHMSJ['J']+'/'\
                                +YmdHMSJ['Y']+YmdHMSJ['J']+"000000.00"+'.'+station+'.'+comp)
        return sacFileNames 
     ```

### 2.2 Run APP <a class="anchor" id="2.2"></a>

The code would run the phase detection using the `detecQuake.getStaL` and associate the detections using `detecQuake.associateSta`. 

The output of the association would use `tool.saveQuakeLs` to append the results in the `work directory` and plot the results using `detecQuake.plotResS`.

In [1]:
# import relavant package
import os
import detecQuake
import trainPSV2 as trainPS
import sacTool
from obspy import UTCDateTime, read
import tool
from locate import locator

##### sac file function
def FileName(net, station, comp, YmdHMSJ):
    """
    Args: 
            net - station network
        station - station name
           comp - station component
        YmdHMSJ - dict of date
    """
    ### Data from external harddisk
    #sac_dir='/Volumes/JW harddisk/Seismology/Data'
    ### Sample data from the computer
    sac_dir = '/Users/jeremy/OneDrive - The Chinese University of Hong Kong/cu/academic/sources/program/seismology/tmp'
    sacFileNames = list()
    c=comp[-1]
    if c=='Z':
        c='U'
    
    # Add location to specific stations
    if net in ['GD']:
        loc = '_00'
        sacFileNames.append(sac_dir+'/'+YmdHMSJ['Y']+'/'+YmdHMSJ['J']+'/'\
                            +YmdHMSJ['Y']+YmdHMSJ['J']+"000000.00"+'.'+station+'.'+comp+loc)
    elif station == 'HKOC':
        loc = '_10'
        sacFileNames.append(sac_dir+'/'+YmdHMSJ['Y']+'/'+YmdHMSJ['J']+'/'\
                            +YmdHMSJ['Y']+YmdHMSJ['J']+"000000.00"+'.'+station+'.'+comp+loc)        
    else:
        sacFileNames.append(sac_dir+'/'+YmdHMSJ['Y']+'/'+YmdHMSJ['J']+'/'\
                            +YmdHMSJ['Y']+YmdHMSJ['J']+"000000.00"+'.'+station+'.'+comp)
    
    #print(sacFileNames)
    return sacFileNames

"""/Users/jeremy/OneDrive - The Chinese University of Hong Kong/cu/academic/sources/program/seismology/tmp/2016/142"""

            
# Generate station list from sacfiles
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
workDir='/Users/jeremy/OneDrive - The Chinese University of Hong Kong/cu/academic/Final Year Project/ML/work/'# workDir: the dir to save the results
staLstFile='station_list'#station list file
bSec=UTCDateTime(2020,1,1).timestamp#begain date
eSec=UTCDateTime(2020,1,14).timestamp#end date
laL=[21, 24]#area: [min_latitude, max_latitude]
loL=[110, 120]#area: [min_longitude, max_longitude]
laN=35#subareas in latitude/the default is enough
loN=35#subareas in longitude
nameFunction=FileName# set to your own file path function

#####no need to change ##########
taupM=tool.quickTaupModel(modelFile='iaspTaupMat')# load pre-calculated travel time result to accelerate the computation speed of travel time 
modelL = [trainPS.loadModel('modelP_320000_100-2-15'),\
trainPS.loadModel('modelS_320000_100-2-15')]#load pre-trained model of P/S
staInfos=sacTool.readStaInfos(staLstFile) #load station information stored in staLstFile
aMat=sacTool.areaMat(laL,loL,laN,loN)#generate subareas according to laL,loL,laN,loN
staTimeML= detecQuake.getStaTimeL(staInfos, aMat, taupM=taupM)#calculate the travel time  range between each station and each subarea
quakeLs=list()# init the quakeLs to store results in memory

for date in range(int(bSec),int(eSec),86400):
    
    date=UTCDateTime(float(date))
    print('pick on ',date)
    staL = detecQuake.getStaL(staInfos, aMat, staTimeML,\
        modelL, date, getFileName=nameFunction,\
        mode='norm',f=[2,15])
    quakeLs.append(detecQuake.associateSta(staL, aMat, \
        staTimeML, timeR=10, maxDTime=3, N=1,locator=\
        locator(staInfos)))
    '''
    save:
    result's in  workDir+'phaseLst'
    result's waveform in  workDir+'output/'
    result's plot picture in  workDir+'output/'
    '''
    tool.saveQuakeLs(quakeLs, workDir+'phaseLst')
    tool.saveQuakeLWaveform(staL, quakeLs[-1], \
        matDir=workDir+'output/',\
            index0=-1500,index1=1500)
    detecQuake.plotResS(staL,quakeLs[-1],outDir=workDir+'output/')
    staL=[]# clear data  to save memory


ModuleNotFoundError: No module named 'h5py'